In [ ]:
#Installing required package
!pip install mlxtend  

In [ ]:
#Libraries for data manipulation
import pandas as pd
import numpy as np

#Library for metrics
from sklearn.metrics import roc_auc_score

#Library for feature selection
from mlxtend.feature_selection import SequentialFeatureSelector

#Library for splitting data
from sklearn.model_selection import train_test_split

In [ ]:
trainCSV = pd.read_csv('./forHiper')
trainCSV.head()

In [ ]:
trainCSV = trainCSV.iloc[:, 4:]
x, y = trainCSV.iloc[:,1:], trainCSV.iloc[:,0]  #X tiene que tener todos los features distintos al target
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 0.25, random_state = 42)

In [ ]:
#Searchs for the best combination of features
#Uses Step Backwards Feature Selection
def findFeatures(xTrain, yTrain, model, k, cv, vb):
    
    feature_selector = SequentialFeatureSelector(model, k_features = k, scoring = 'roc_auc',\
                                                 cv = cv, verbose = vb, forward = False)
    features = feature_selector.fit(xTrain, yTrain) 
    return xTrain.columns[list(features.k_feature_idx_)]

## Random Forest

https://stackabuse.com/applying-wrapper-methods-in-python-for-feature-selection/

In [ ]:
from sklearn.ensemble import RandomForestClassifier

#hypermarameters = getHyperparameters('randomForest')

#Construir RF con parametros conseguidos en getHyperparameters
RF = RandomForestClassifier()
bestParametersRF = findFeatures(xTrain, yTrain, RF, 15, 4, 2) #bestParametersRF
bestParametersRF

## KNN

http://rasbt.github.io/mlxtend/user_guide/feature_selection/SequentialFeatureSelector/

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

#hypermarameters = getHyperparameters('knn')

#Construir RF con parametros conseguidos en getHyperparameters
knn = KNeighborsClassifier()
bestParametersKNN = findFeatures(xTrain, yTrain, knn, 3, 4, 2) #bestParametersRF
bestParametersKNN

## Decision Tree Classifier

In [ ]:
from sklearn import tree

#hypermarameters = getHyperparameters('decissionTreClassifier')

#Construir Decission Tree con parametros conseguidos en getHyperparameters
DTC = tree.DecisionTreeClassifier(min_samples_split = 10)
bestParametersDTC = findFeatures(xTrain, yTrain, DTC, 15, 3, 1)
bestParametersDTC

## XGBoost

https://machinelearningmastery.com/feature-importance-and-feature-selection-with-xgboost-in-python/#:~:text=Feature%20Importance%20and%20Feature%20Selection%20With%20XGBoost%20in%20Python,-By%20Jason%20Brownlee&text=A%20benefit%20of%20using%20ensembles,from%20a%20trained%20predictive%20model.

https://scikit-learn.org/dev/auto_examples/feature_selection/plot_select_from_model_diabetes.html

In [ ]:
from xgboost import XGBClassifier
from matplotlib import pyplot


#hypermarameters = getHyperparameters('xgboost')

#Construir XBBoost con parametros conseguidos en getHyperparameters

model = XGBClassifier()
model.fit(xTrain, yTrain)

#Plot features and  get the important ones
print(model.feature_importances_)
# plot feature importance
# select features using threshold
selection = SelectFromModel(model, threshold=thresh, prefit=True)
select_X_train.fit(xTrain, yTrain)
select_X_train = selection.transform(X_train)
n_features = select_X_train.transform(xTrain).shape[1]
n_features